# Lab | Web Scraping

Welcome to the IMDb Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the vast sea of data available on IMDb, one of the largest online databases of movie, TV, and celebrity information. As budding data scientists and business analysts, you have been tasked to scrape a specific subset of data from IMDb to assist film production companies in understanding the landscape of highly-rated movies in a defined time period. Your insights will potentially influence the making of the next netflix movie!

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The entertainment industry, being no exception, utilizes data analytics to comprehend market trends, audience preferences, and the performance of films based on various parameters such as director, genre, stars involved, etc. IMDb stands as a goldmine of such data, offering intricate details of almost every movie ever made.

**Task**

Your task is to create a Python script using `BeautifulSoup` and `pandas` to scrape IMDb movie data based on user ratings and release dates. This script should be able to filter movies with ratings above a certain threshold and within a specified date range.

**Expected Outcome**

- A function named `scrape_imdb` that takes four parameters: `title_type`,`user_rating`, `start_date`, and `end_date`.
- The function should return a DataFrame with the following columns:
  - **Movie Nr**: The number representing the movie’s position in the list.
  - **Title**: The title of the movie.
  - **Year**: The year the movie was released.
  - **Rating**: The IMDb rating of the movie.
  - **Runtime (min)**: The duration of the movie in minutes.
  - **Genre**: The genre of the movie.
  - **Description**: A brief description of the movie.
  - **Director**: The director of the movie.
  - **Stars**: The main stars of the movie.
  - **Votes**: The number of votes the movie received.
  - **Gross ($M)**: The gross earnings of the movie in millions of USD.

You will execute this script to scrape data for movies with the Title Type `Feature Film` that have a user rating of `7.5 and above` and were released between `January 1, 1990, and December 31, 1992`.

Remember to experiment with different title types, dates and ratings to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [IMDb Advanced Search](https://www.imdb.com/search/title/)


**Hint**

Your first mission is to familiarize yourself with the IMDb advanced search page. Head over to [IMDb advanced search](https://www.imdb.com/search/title/) and input the following parameters, keeping all other fields to their default values or blank:

- **Title Type**: Feature film
- **Release date**: From 1990 to 1992 (Note: You don't need to specify the day and month)
- **User Rating**: 7.5 to -

Upon searching, you'll land on a page showcasing a list of movies, each displaying vital details such as the title, release year, and crew information. Your task is to scrape this treasure trove of data.

Carefully examine the resulting URL and construct your own URL to include all the necessary parameters for filtering the movies.


---

**Best of luck! Immerse yourself in the world of movies and may the data be with you!**

**Important note**:

In the fast-changing online world, websites often get updates and make changes. When you try this lab, the IMDb website might be different from what we expect.

If you run into problems because of these changes, like new rules or things that stop you from getting data, don't worry! Instead, get creative.

You can choose another website that interests you and is good for scraping data. Websites like Wikipedia or The New York Times are good options. The main goal is still the same: get useful data and learn how to scrape it from a website that you find interesting. It's a chance to practice your web scraping skills and explore a source of information you like.

In [ ]:
# Your solution goes here
import requests
from bs4 import BeautifulSoup
import pandas as pd

# url = "https://www.decathlon.com/collections/mountain-bikes"
# response = requests.get(url)
# soup = BeautifulSoup(response.content, "html.parser")

In [ ]:
import re

In [ ]:
def scrape_imdb(title_type, user_rating, start_date, end_date):
    
    url = f"https://www.imdb.com/search/title/?title_type={title_type}&user_rating={user_rating}&release_date={start_date},{end_date}"
    # url = 'https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,10'
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(url, headers=headers)
    # response = requests.get(url)
    print(response)
    soup = BeautifulSoup(response.text, 'html.parser')

    movies = soup.find_all('li', class_='ipc-metadata-list-summary-item')

    return movies

# Teste
result = scrape_imdb('feature', 7.5, '1990-01-01', '1992-12-31')

In [ ]:
data = {
        'Movie_Nr': [],
        'Title': [],
        'Year': [],
        'Rating': [],
        'Runtime': [],
        'Genre': [],
        'Description': [],
        'Director': [],
        'Stars': [],
        'Votes': [],
        'Gross ($M)': []
    }
datac = pd.DataFrame(data)

In [ ]:
def scrape_content(movies,data):
    for i, movie in enumerate(movies):

        pattern = r'(\d+\.)'
        pattern_title = r"([^0-9.\s].+)"

        number = re.search(pattern, movie.find('h3').text).group(1)
        title = re.search(pattern_title, movie.find('h3').text).group(1)
        meta_Y_H_R = movie.find_all(name='span', class_='sc-b189961a-8 kLaxqf dli-title-metadata-item')
        # hours = re.search(r'\dh(\s\d+m)?', meta_Y_H_R[4:])
        stars = movie.find('span', class_='ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating').get('aria-label')
        vote = movie.find('span', class_='ipc-rating-star--voteCount').text
        Description = movie.find('div', class_='ipc-html-content ipc-html-content--base sc-74bf520e-0 iiRKKi dli-plot-container').text
        
        data.loc[i,'Movie_Nr'] = number[:-1]
        data.loc[i,'Title'] = title
        data.loc[i, 'Year'] = meta_Y_H_R[0].text
        data.loc[i,'Runtime'] = meta_Y_H_R[1].text
        data.loc[i,'Rating'] = meta_Y_H_R [2].text
        data.loc[i,'Stars'] = stars[13:]
        data.loc[i,'Votes'] = vote
        data.loc[i,'Description'] = Description

        # data['Genre'].append(movie.find('span', class_='genre').text.strip())        
        # data['Director'].append(movie.find_all('p', class_='')[1].a.text)
        # stars = [star.text for star in movie.find_all('p', class_='')[1].find_all('a')[1:]]
        # data['Stars'].append(', '.join(stars))
        # try:
        #     data['Gross ($M)'].append(float(movie.find('p', class_='sort-num_votes-visible').find_all('span', attrs={'name': 'nv'})[1]['data-value']) / 1000000)
        # except IndexError:
        #     data['Gross ($M)'].append(None)

        i +=1
    return data


In [463]:
#teste 2
result2 = scrape_content(result, datac)
print(result2)


   Movie_Nr                                         Title  Year     Rating  \
0         1                                    Goodfellas  1990          R   
1         2                      The Silence of the Lambs  1991          R   
2         3                    Terminator 2: Judgment Day  1991          R   
3         4                                Reservoir Dogs  1992          R   
4         5                          Fried Green Tomatoes  1991      PG-13   
5         6                        The Godfather Part III  1990          R   
6         7                                  Total Recall  1990          R   
7         8                                    Home Alone  1990         PG   
8         9                      The Last of the Mohicans  1992          R   
9        10                            Dances with Wolves  1990      PG-13   
10       11                                    Unforgiven  1992          R   
11       12                              Scent of a Woman  1992 

## BONUS

The search results span multiple pages, housing a total of 631 movies in our example with each page displaying 50 movies at most. To scrape data seamlessly from all pages, you'll need to dive deep into the structure of the URLs generated with each "Next" click.

Take a close look at the following URLs:
- First page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,
  ```
- Second page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=51&ref_=adv_nxt
  ```
- Third page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=101&ref_=adv_nxt
  ```

You should notice a pattern. There is a `start` parameter incrementing by 50 with each page, paired with a constant `ref_` parameter holding the value "adv_nxt".

Modify your script so it's capable of iterating over all available pages to fetch data on all the 631 movies (631 is the total number of movies in the proposed example).

In [ ]:
# Your solution goes here